In [1]:
from bertviz import head_view
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import random
import re
import os
from sklearn.metrics import classification_report

In [2]:
def get_questions(path):
    '''
        Read in all the questions with their ids from a specific path
    '''

    questions = {}
    for root, dirs, files in os.walk(path):
        for file in files:
            if file=='context.txt':
                with open(os.path.join(root,file)) as f:
                    q_full = f.read().split('<br>')
                    q_text = q_full[2]
                    q_title = q_full[0]
                    q_context = ' '.join([x for x in re.split("[?.!;]", q_text) if x!=""][-3:])
                    # Strip off newline and tab characters
                    q_context = q_context.replace('\n', '').replace('\t', '')
                    q_context = q_context + " " + q_title
                    questions[root.split('/')[-1]] = q_context

    return questions

In [15]:
def find_reply(ind, df):
    '''
    Return a reliable context for every sentence, instead of one of 3 possible
    things which might confuse the downstream model
    '''

    split_ind = ind.split('-')
    context = df.loc[(df['Post.ID']==split_ind[0]) & \
                     (df['Reply.ID']==split_ind[1]), 'Sentence'].values.tolist()
    context = " ".join(context)

    return context

In [16]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

In [17]:
# Set random seeds for reproducibility on a specific machine
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.manual_seed(1)
torch.cuda.manual_seed(1)
random.seed(1)
np.random.seed(1)
np.random.RandomState(1)

RandomState(MT19937) at 0x12505E150

In [24]:
pd.set_option('display.max_colwidth', None)

data = 'needadvice'

pred_file = '../../preds/classifier_needadvice_bert_dropout:0.1_lr_tr:1e-05_lr_cl:1e-05_wd:0_batch:32_finetune:True_query:False_context:False_seed:20_multigpu:True_labels:ds_frac:1.csv'

dev = pd.read_csv(pred_file, header=0, sep='\t')
dev_sentences = dev['Sentence'].tolist()
dev_labels_DS = dev['DS_Label'].values
dev_labels_Maj = dev['Majority_label'].values
dev['Pred_Label'] = dev['Pred_Label'].astype(int)
dev['Reply.ID'] = dev['Reply.ID'].astype(int).astype(str)
dev_preds = dev['Pred_Label'].values
dev = dev.set_index('ID')
questions = get_questions('../../rawdata/' + data)
dev['Question'] = dev.index.map(lambda x:questions[x.split('-')[0]])
dev['Reply'] = dev.index.map(lambda x: find_reply(x, dev))

In [25]:
dev[(dev['DS_Label']==1)& (dev['Pred_Label']==1)].sample(50)

,Majority_label,DS_Label,Sentence,Flair,Post.ID,Reply.ID,Sent.Num,Pred_Label,Question,Reply
ID,,,,,,,,,,
d3cklw-13-0,1,1,"Get a plant , an orchid or something you need to look after .",Mental Health,d3cklw,13,0,1,[removed] [Lonely and Depressed] In Mental Institution with No Chance of Leaving,"Get a plant , an orchid or something you need to look after ."
afzgo6-3-0,1,1,You should have no problem using images from the web that are owned by other people as long as A. you are not selling or gaining financial profit for using someone else 's image and B. you are able to cite the original source of the image .,Education,afzgo6,3,0,1,I'm making a powerpoint presentation for school and if possible I would use a certain picture from gettyimages Is it possible to use the pics with the watermarks on them for free And by the way does anybody know how to get stockphotos for free Can I just use pictures from for example gettyimages with the watermark on them?,You should have no problem using images from the web that are owned by other people as long as A. you are not selling or gaining financial profit for using someone else 's image and B. you are able to cite the original source of the image .
cqwmxj-1-2,1,1,No matter your choice a quick internet search to find out if you need school to do what you want .,Life Decisions,cqwmxj,1,2,1,What are your experiences with college or getting a good job after high school I’m so stressed I’m sick Heellp I’m trying to figure out if I should go to college or just find a good job...,Take general studies for a year and see if anything pulls you or perhaps get a technical degree . You can definitely find something without college but it sounds like you want to go either way . I think that is one of the most stressful fields . I think you need to know firstly that if you are sick stressed just trying to make the choice of college or not ; would emergency medicine even be a good career choice for you ? No matter your choice a quick internet search to find out if you need school to do what you want .
bmqfe9-16-6,1,1,My advice is to ween off of whatever you 've been doing in your past time and adventure into a territory of hobby you have n't or too afraid to explore .,Mental Health,bmqfe9,16,6,1,"nothing drastic has happened to me recently, i have no idea how to get my motivation to do my hobbies or other things again, ive watched motivation videos before and they dont really help EDIT: I read all your replies and i see i aint alone here for sure will definitely take your advice and see if that helps, hopefully it makes me happier Slowly getting disinterested in everything","It 's how we prove our skill , and what we can achieve . The reason we do things is to make ourselves feel proud of what we , as a human , can accomplish . My advice is to ween off of whatever you 've been doing in your past time and adventure into a territory of hobby you have n't or too afraid to explore . It 's the sense of victory over what CAN be done . Much love you you , and I hope this helped <3 Maybe this'll give you that sense of interest and achievement you 're looking for . If we repetiviley do the same thing , we 'll eventually think , "" Well , I 've felt this type of success before . and well .... maybe we wo n't feel so proud of it the other time around . PLENTY of times actually . """
bmqfe9-6-3,0,1,Or try volunteering !,Mental Health,bmqfe9,6,3,1,"nothing drastic has happened to me recently, i have no idea how to get my motivation to do my hobbies or other things again, ive watched motivation videos before and they dont really help EDIT: I read all your replies and i see i aint alone here for sure will definitely take your advice and see if that helps, hopefully it makes me happier Slowly getting disinterested in everything",Or try volunteering ! Yep . You might have simply changed as a person and your old hobbies are n’t “ you ” anymore . Try doing some things you never have done before . Sometimes this is just an ind

In [26]:
dev.loc[(dev['Post.ID']=='cgtyv4') & (dev['Reply.ID']=='6'), :]

,Majority_label,DS_Label,Sentence,Flair,Post.ID,Reply.ID,Sent.Num,Pred_Label,Question,Reply
ID,,,,,,,,,,
cgtyv4-6-0,1,1,Give yourself a budget and require yourself to waste it on frivolous things .,Life Decisions,cgtyv4,6,0,1,And a part of myself hates the feeling of being frugal cause I’ve never allowed myself to have nice things I’m going to be that person who dies and never had the chance to treat myself out How do I get over this and occasionally tell myself it’s okay to spend a little on myself Having a hard time spending money on myself (32F),Give yourself a budget and require yourself to waste it on frivolous things . That can actually be very therapeutic but wo n't take things overboard . Goodbudget is a great app to help with this .
cgtyv4-6-1,0,0,That can actually be very therapeutic but wo n't take things overboard .,Life Decisions,cgtyv4,6,1,0,And a part of myself hates the feeling of being frugal cause I’ve never allowed myself to have nice things I’m going to be that person who dies and never had the chance to treat myself out How do I get over this and occasionally tell myself it’s okay to spend a little on myself Having a hard time spending money on myself (32F),Give yourself a budget and require yourself to waste it on frivolous things . That can actually be very therapeutic but wo n't take things overboard . Goodbudget is a great app to help with this .
cgtyv4-6-2,0,0,Goodbudget is a great app to help with this .,Life Decisions,cgtyv4,6,2,1,And a part of myself hates the feeling of being frugal cause I’ve never allowed myself to have nice things I’m going to be that person who dies and never had the chance to treat myself out How do I get over this and occasionally tell myself it’s okay to spend a little on myself Having a hard time spending money on myself (32F),Give yourself a budget and require yourself to waste it on frivolous things . That can actually be very therapeutic but wo n't take things overboard . Goodbudget is a great app to help with this .


In [27]:
dev.loc[(dev['Post.ID']=='c8tmtz') & (dev['Reply.ID']=='1'), :]

,Majority_label,DS_Label,Sentence,Flair,Post.ID,Reply.ID,Sent.Num,Pred_Label,Question,Reply
ID,,,,,,,,,,
c8tmtz-1-1,0,0,I did n't even know I was interested in some of my favourite hobbies until a few years ago .,Education,c8tmtz,1,1,0,I’m pretty skinny and about average-ish height My stamina is terrible since I don’t do exercise so I’m not sure if I’ll be good enough to even start doing something like a physical activity Please help Is it too late to start a hobby/activity at 12?,I did n't even know I was interested in some of my favourite hobbies until a few years ago . Go do the thing OP ! You can start any hobby at any age .
c8tmtz-1-2,1,1,Go do the thing OP !,Education,c8tmtz,1,2,1,I’m pretty skinny and about average-ish height My stamina is terrible since I don’t do exercise so I’m not sure if I’ll be good enough to even start doing something like a physical activity Please help Is it too late to start a hobby/activity at 12?,I did n't even know I was interested in some of my favourite hobbies until a few years ago . Go do the thing OP ! You can start any hobby at any age .
c8tmtz-1-0,0,0,You can start any hobby at any age .,Education,c8tmtz,1,0,0,I’m pretty skinny and about average-ish height My stamina is terrible since I don’t do exercise so I’m not sure if I’ll be good enough to even start doing something like a physical activity Please help Is it too late to start a hobby/activity at 12?,I did n't even know I was interested in some of my favourite hobbies until a few years ago . Go do the thing OP ! You can start any hobby at any age .


In [ ]:
dev[(dev['DS_Label']==1) & (dev['Pred_Label']==0)].sample(50)

In [ ]:
dev[(dev['DS_Label']==0) & (dev['Pred_Label']==1)].sample(50)

## Model loading

In [ ]:
saved_model = '../../saved_pretrained/classifier_needadvice_bert_dropout:0.1_lr_tr:1e-05_lr_cl:1e-05_wd:0_batch:32_finetune:True_query:False_context:False_seed:45_multigpu:True_labels:ds_frac:1/'

tokenizer = AutoTokenizer.from_pretrained(saved_model)
model = AutoModelForSequenceClassification.from_pretrained(saved_model)

model.eval()

In [ ]:
def predict(sentence):
    '''
        Returns classification value - 0,1 given sentence
    '''
    
    input_ids = torch.tensor(tokenizer.encode(sentence)).unsqueeze(0)
#     print(input_ids)
    with torch.no_grad():
        logits = model(input_ids)[0]
    print(logits)

In [ ]:
print(dev.loc['c823ha-7-6','Sentence'])

In [ ]:
predict(dev.loc['c823ha-7-6	', 'Sentence'])

## Attention Heads View

In [ ]:
def show_head_view(model, tokenizer, sentence_a, sentence_b=None):
    inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True, return_token_type_ids=True)
    input_ids = inputs['input_ids']
    if sentence_b:
        token_type_ids = inputs['token_type_ids']
        with torch.no_grad():
            attention = model(input_ids, token_type_ids=token_type_ids)[-1]
        import ipdb; ipdb.set_trace()
        sentence_b_start = token_type_ids[0].tolist().index(1)
    else:
        with torch.no_grad():
            attention = model(input_ids)[-1]
        sentence_b_start = None
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)    
    head_view(attention, tokens, sentence_b_start)

In [ ]:
sentence = dev.loc['c823ha-7-6	']['Sentence']

In [ ]:
show_head_view(model, tokenizer, sentence_a=sentence)

# Personal narrative analysis 

In [52]:
test2 = pd.read_csv('../../preds/test/classifier_needadvice_bert_dropout:0.1_lr_tr:1e-05_lr_cl:1e-05_wd:0_batch:32_finetune:True_query:False_context:False_seed:2_multigpu:True_labels:ds_frac:1_TEST.csv', sep='\t', header=0)
test2['Sentence'] = test2['Sentence'].str.lower()
test2_personal = test2[test2['Sentence'].str.contains(' me ') | test2['Sentence'].str.contains(' my ') | test2['Sentence'].str.contains(' we ')]

test13 = pd.read_csv('../../preds/test/classifier_needadvice_bert_dropout:0.1_lr_tr:1e-05_lr_cl:1e-05_wd:0_batch:32_finetune:True_query:False_context:False_seed:13_multigpu:True_labels:ds_frac:1_TEST.csv', sep='\t', header=0)
test13['Sentence'] = test13['Sentence'].str.lower()
test13_personal = test13[test13['Sentence'].str.contains(' me ') | test13['Sentence'].str.contains(' my ') | test13['Sentence'].str.contains(' we ')]

test20 = pd.read_csv('../../preds/test/classifier_needadvice_bert_dropout:0.1_lr_tr:1e-05_lr_cl:1e-05_wd:0_batch:32_finetune:True_query:False_context:False_seed:20_multigpu:True_labels:ds_frac:1_TEST.csv', sep='\t', header=0)
test20['Sentence'] = test20['Sentence'].str.lower()
test20_personal = test20[test20['Sentence'].str.contains(' me ') | test20['Sentence'].str.contains(' my ') | test20['Sentence'].str.contains(' we ')]

test45 = pd.read_csv('../../preds/test/classifier_needadvice_bert_dropout:0.1_lr_tr:1e-05_lr_cl:1e-05_wd:0_batch:32_finetune:True_query:False_context:False_seed:45_multigpu:True_labels:ds_frac:1_TEST.csv', sep='\t', header=0)
test45['Sentence'] = test45['Sentence'].str.lower()
test45_personal = test45[test45['Sentence'].str.contains(' me ') | test45['Sentence'].str.contains(' my ') | test45['Sentence'].str.contains(' we ')]

test78 = pd.read_csv('../../preds/test/classifier_needadvice_bert_dropout:0.1_lr_tr:1e-05_lr_cl:1e-05_wd:0_batch:32_finetune:True_query:False_context:False_seed:78_multigpu:True_labels:ds_frac:1_TEST.csv', sep='\t', header=0)
test78['Sentence'] = test78['Sentence'].str.lower()
test78_personal = test78[test78['Sentence'].str.contains(' me ') | test78['Sentence'].str.contains(' my ') | test78['Sentence'].str.contains(' we ')]

In [53]:
print(classification_report(test2_personal['DS_Label'], test2_personal['Pred_Label'], digits=4))

              precision    recall  f1-score   support

           0     0.8488    0.9241    0.8848        79
           1     0.5000    0.3158    0.3871        19

    accuracy                         0.8061        98
   macro avg     0.6744    0.6199    0.6360        98
weighted avg     0.7812    0.8061    0.7883        98



In [54]:
print(classification_report(test13_personal['DS_Label'], test13_personal['Pred_Label'], digits=4))

              precision    recall  f1-score   support

           0     0.8537    0.8861    0.8696        79
           1     0.4375    0.3684    0.4000        19

    accuracy                         0.7857        98
   macro avg     0.6456    0.6272    0.6348        98
weighted avg     0.7730    0.7857    0.7785        98



In [55]:
print(classification_report(test20_personal['DS_Label'], test20_personal['Pred_Label'], digits=4))

              precision    recall  f1-score   support

           0     0.8721    0.9494    0.9091        79
           1     0.6667    0.4211    0.5161        19

    accuracy                         0.8469        98
   macro avg     0.7694    0.6852    0.7126        98
weighted avg     0.8323    0.8469    0.8329        98



In [56]:
print(classification_report(test45_personal['DS_Label'], test45_personal['Pred_Label'], digits=4))

              precision    recall  f1-score   support

           0     0.8795    0.9241    0.9012        79
           1     0.6000    0.4737    0.5294        19

    accuracy                         0.8367        98
   macro avg     0.7398    0.6989    0.7153        98
weighted avg     0.8253    0.8367    0.8291        98



In [57]:
print(classification_report(test78_personal['DS_Label'], test78_personal['Pred_Label'], digits=4))

              precision    recall  f1-score   support

           0     0.8571    0.9873    0.9176        79
           1     0.8571    0.3158    0.4615        19

    accuracy                         0.8571        98
   macro avg     0.8571    0.6516    0.6896        98
weighted avg     0.8571    0.8571    0.8292        98



In [12]:
test2_personal

,ID,Majority_label,DS_Label,Sentence,Flair,Post.ID,Reply.ID,Sent.Num,Pred_Label
3,cnyd1c-2-2,0,1,"In general , I found answering questions as if...",Education,cnyd1c,2.0,2,0.0
9,bwnlrj-9-1,0,0,Have you ever been evaluated by a psychiatrist...,Career,bwnlrj,9.0,1,0.0
15,cefcvm-3-2,1,1,"In the meantime , I strongly urge you to find ...",Career,cefcvm,3.0,2,1.0
17,e4a5hu-2-1,1,1,"If it is something you are comfortable with , ...",Education,e4a5hu,2.0,1,1.0
27,bkksjn-33-0,0,0,"a close friend of mine showed me her scars , I...",Mental Health,bkksjn,33.0,0,0.0
...,...,...,...,...,...,...,...,...,...
864,ac9n9g-11-0,0,0,The dentists I ’ve seen after twice long negle...,Life Decisions,ac9n9g,11.0,0,0.0
870,bocbvr-4.1-4,0,0,Mostly because I ’m doing all the work ..,Career,bocbvr,4.1,4,0.0
874,bwnlrj-10.1-2,0,0,"Like I will still be myself , but when I ’m wi...",Career,bwnlrj,10.1,2,0.0
890,annlx1-7-1,1,1,"Exercise your body and mind , eat well , do n’...",Mental Health,annlx1,7.0,1,1.0


In [34]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

dev['Sentence'] = dev['Sentence'].str.lower()
dev_personal = dev[dev['Sentence'].str.contains(' we ') | dev['Sentence'].str.contains(' me ') | dev['Sentence'].str.contains(' my ')]

dev_personal

,Majority_label,DS_Label,Sentence,Flair,Post.ID,Reply.ID,Sent.Num,Pred_Label,Question,Reply
ID,,,,,,,,,,
bm6tj7-4-0,0,0,"it helped my daughter , i hope you get the chance to do it too .",Life Decisions,bm6tj7,4,0,0,I don't know what college should I go to or what my dream job is I only know that I have to choose one thing and focus on it How can I do that Help me finding my life goal,"It helped my daughter , I hope you get the chance to do it too . Good luck"
bzp12f-14-0,0,0,"“ death does n’t discriminate between the sinners and the saints , it takes and takes , but we keep living anyway , we rise , fall , break and make mistakes ; but if there ’s a reason why i ’m still alive when so few survive , i ’m willing to wait for it .",Mental Health,bzp12f,14,0,0,"I’m just so scared, and I’m not even sure if this fear is rational I would just like to know what the next step is for getting out of this hole of depression, because I really do want to get better EDIT: I just went outside and took a walk, and then went on a bike ride (I’m in Japan rn) and I feel so much better, like my normal self I’m scared of death","“ Death does n’t discriminate between the sinners and the saints , it takes and takes , but we keep living anyway , we rise , fall , break and make mistakes ; but if there ’s a reason why I ’m still alive when so few survive , I ’m willing to wait for it . ” - One boi stuck outside the room where it happens"
abwcwf-1-4,0,0,edit : i should mention that my high school gpa was n’t the greatest .,Education,abwcwf,1,4,0,"I have multiple auditions at multiple colleges lined up, but I fear because of my GPA I won’t get in Should I just give up and go to community college Would it be better to rebuild my GPA then transfer Do I go to community college?","Edit : I should mention that my high school GPA was n’t the greatest . Academically wise , it ’s slightly harder than high school yet a bit easier than university . It was nearly a 2.8 . Ended community college with a 3.6 . There ’s not much of a downside minus your social life if you care too much about it . I was accepted into some decent universities , but I opted to enter community college instead . Saved money , and it ’ll allow yourself some time to figure out what you truly want to do ."
axmflf-12-5,0,0,"i have my own well being and peace of mind to care for and i ca n't be her babysitter any longer and i ca n't apologize for it , because she 's bringing me down under with her .",Friendships,axmflf,12,5,0,"I do want him to be happy, but I'm *his* only friend too What I need advice on is this: how to I cut ties with someone who cares deeply about me, but that is more of a nuisance to me than somebody I love I know that I may get down voted for being an insensitive prick, but I just want to know how to end a friendship without hurting him any more My best friend hates me and I don't care. How should I handle this?","I have my own well being and peace of mind to care for and I ca n't be her babysitter any longer and I ca n't apologize for it , because she 's bringing me down under with her . She was in a bad space mentally and refused any help , and refused to do anything to help herself . Best of luck I ve gone through nearly the exact situation . She also was kind of a selfish and shitty friend . Eventually she asked me if we were still friends and I explained to her why it 's not the best thing for me moving forward . Just let him know that you appreciate and cherish his friendship and now its time for both of you to grow . I kinda Iced her out for a few months because I resented her , and we used to spend like everyday together ."
c8tmtz-3-2,0,1,"based on my experience , just do it :)",Education,c8tmtz,3,2,0,I’m pretty skinny and about average-ish height My stamina is terrible since I don’t do exercise so I’m not sure if I’ll be good enough to even start doing something like a physical activity Please help Is it too late to start a hobby/activity at 12?,"based on my experien

In [ ]:
1234567891011121314151617181920212223242526272829303132333435363738394041424344454647484950515253545556575859

In [36]:
from sklearn.metrics import classification_report
print(classification_report(dev_personal['DS_Label'], dev_personal['Pred_Label']))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93        89
           1       0.45      0.45      0.45        11

    accuracy                           0.88       100
   macro avg       0.69      0.69      0.69       100
weighted avg       0.88      0.88      0.88       100



In [37]:
dev_personal_i = dev[dev['Sentence'].str.contains(' i ')]

print(classification_report(dev_personal_i['DS_Label'], dev_personal_i['Pred_Label']))

              precision    recall  f1-score   support

           0       0.85      0.92      0.89       105
           1       0.62      0.43      0.51        30

    accuracy                           0.81       135
   macro avg       0.73      0.68      0.70       135
weighted avg       0.80      0.81      0.80       135



In [58]:
dev_personal.shape

(100, 10)

In [59]:
dev_personal['Sentence'].tolist()

['it helped my daughter , i hope you get the chance to do it too .',
 '“ death does n’t discriminate between the sinners and the saints , it takes and takes , but we keep living anyway , we rise , fall , break and make mistakes ; but if there ’s a reason why i ’m still alive when so few survive , i ’m willing to wait for it .',
 'edit : i should mention that my high school gpa was n’t the greatest .',
 "i have my own well being and peace of mind to care for and i ca n't be her babysitter any longer and i ca n't apologize for it , because she 's bringing me down under with her .",
 'based on my experience , just do it :)',
 'plenty of kids at my boarding schools came only for grade 12',
 'the 1st week was grueling for me but afterwards it was like i had a weird brain reset .',
 'the wording you used earlier ( rough paraphrasing ) " moving into my own place would make me happier " i think you should shift your focus on being content with your current situation and things may improve for 